# Intro to gragh convolutional surrogate models
Eamon Whalen

In [1]:
import sys
import numpy as np
import pandas as pd
import altair as alt

from gcnSurrogate.models.feastnetSurrogateModel import FeaStNet
from gcnSurrogate.readers.loadConmechGraphs import loadConmechGraphs
from gcnSurrogate.visualization.altTrussViz import plotTruss, interactiveErrorPlot
from gcnSurrogate.util.gcnSurrogateUtil import *

## 1. Load simulation data

In [2]:
dataDir = 'data/2D_Truss_v1.3/conmech/design_7_N_1000/'
allGraphsUnfiltered = loadConmechGraphs(dataDir)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphsUnfiltered]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,1000.000000
mean,0.137562
std,1.740254
min,0.004530
25%,0.010635
50%,0.016778
75%,0.032392
max,46.419552


In [3]:
plotTruss(allGraphsUnfiltered[0], showDeformed=True, defScale=100)

alt.LayerChart(...)

## 2. Filter and partition

In [4]:
allGraphs = filterbyDisp(allGraphsUnfiltered, 0.9)
trainData, valData, testData = partitionGraphList(allGraphs)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphs]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,900.000000
mean,0.019962
std,0.013149
min,0.004530
25%,0.010309
50%,0.014975
75%,0.026453
max,0.067472


## 3. Train a GCN

In [5]:
gcn = FeaStNet()
history = gcn.trainModel(trainData, valData, 
                         epochs=100,
                         saveDir='results/gcn03/')

plotHistory(history)

epoch: 0   trainLoss: 9.6764e-01   valLoss:9.7939e-01  time: 1.90e+00
epoch: 1   trainLoss: 7.7749e-01   valLoss:9.4753e-01  time: 1.54e+00
epoch: 2   trainLoss: 6.3086e-01   valLoss:9.9381e-01  time: 1.55e+00
epoch: 3   trainLoss: 5.6648e-01   valLoss:1.1048e+00  time: 1.54e+00
epoch: 4   trainLoss: 4.6124e-01   valLoss:1.0929e+00  time: 1.54e+00
epoch: 5   trainLoss: 3.9931e-01   valLoss:1.4213e+00  time: 1.56e+00
epoch: 6   trainLoss: 3.4580e-01   valLoss:1.9574e+00  time: 1.55e+00
epoch: 7   trainLoss: 3.2430e-01   valLoss:1.4722e+00  time: 1.55e+00
epoch: 8   trainLoss: 2.5445e-01   valLoss:1.0620e+00  time: 1.55e+00
epoch: 9   trainLoss: 2.3006e-01   valLoss:9.0022e-01  time: 1.54e+00
epoch: 10   trainLoss: 1.7727e-01   valLoss:9.0577e-01  time: 1.57e+00
epoch: 11   trainLoss: 1.7046e-01   valLoss:3.6550e-01  time: 1.58e+00
epoch: 12   trainLoss: 1.7414e-01   valLoss:1.1265e+00  time: 1.56e+00
epoch: 13   trainLoss: 1.4438e-01   valLoss:5.5349e-01  time: 1.57e+00
epoch: 14   trai

alt.Chart(...)

## 4. Test the GCN

In [6]:
trainRes = gcn.testModel(trainData)
testRes = gcn.testModel(testData)
pd.DataFrame([trainRes, testRes], index=['train', 'test'])

,mse,mae,mre,peakR2,maxAggR2,meanAggR2,minAggR2
train,0.000004,0.001379,0.073992,0.935180,0.974629,0.848511,0.0
test,0.000007,0.001615,0.084994,0.801703,0.932201,0.797055,0.0


## 5. Visualize some predictions

In [9]:
i = 9
pred = gcn.predict([testData[i]])[0]
plotTruss(testData[i], showDeformed=True, defScale=200, prediction=pred)

alt.LayerChart(...)

In [8]:
# interactive scatter plot
alt.data_transformers.enable('json')
allPreds = gcn.predict(testData)
display(interactiveErrorPlot(testData, allPreds))
alt.data_transformers.enable('default');

alt.HConcatChart(...)